## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


Student name:Myra Kadenge
Student pace: full time
Scheduled project review date/time:
Instructor name:
Blog post URL:


# MICROSOFT NEW STUDIO PROJECT ANALYSIS

# Overview

Microsoft is a multinational technological company 
As a private company and knowing that they want to open a new studio, their maon concern should not only be generating revenue but profits. This thus poses a challenge for the company which is to figure out how to open a profitable studio and explore the films that are currently doing the best, moreover finding a unique set of films that could give them a unique identity. The most essential thing to do would be to look at the most profitable movies.


# Data Analysis

In [3]:
#Start by importing the standard packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

%matplotlib inline

In [22]:
#First load all the data sets to understand the different elements in each and how they may interract with each 
#other to help in deciding which ones to merge as well
df_bom_movies = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
df_rt = pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter = '\t')
df_budget = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
df4 = pd.read_csv('zippedData/tmdb.movies.csv.gz')


In [23]:
conn = sqlite3.connect('zippedData/im.db')
cur = conn.cursor()


In [6]:
#Looking at the first data set bom.movies
df_bom_movies
df_bom_movies.head(10)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
5,The Twilight Saga: Eclipse,Sum.,300500000.0,398000000,2010
6,Iron Man 2,Par.,312400000.0,311500000,2010
7,Tangled,BV,200800000.0,391000000,2010
8,Despicable Me,Uni.,251500000.0,291600000,2010
9,How to Train Your Dragon,P/DW,217600000.0,277300000,2010


In [7]:
#get a summary of the data to see if there could be any missing values
df_bom_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [8]:
#studio, domestic gross and foreign gross seem to have missing values
df_bom_movies.isna().sum()

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

In [9]:
df_bom_movies.duplicated().value_counts()#to check for any duplicates
#we have no duplicates in this dataset

False    3387
dtype: int64

In [10]:
#I will convert the foreign gross data to numeric data
# Clean the 'foreign_gross' column by removing commas
df_bom_movies['foreign_gross'] = df_bom_movies['foreign_gross'].str.replace(',', '')

# Convert the 'foreign_gross' column to numeric
df_bom_movies['foreign_gross'] = pd.to_numeric(df_bom_movies['foreign_gross'], errors='coerce')

# Display non-numeric values (if any)
non_numeric_values = df_bom_movies['foreign_gross'][df_bom_movies['foreign_gross'].isna()]
print("Non-numeric values in 'foreign_gross':")
print(non_numeric_values)

# Display the first few rows of the DataFrame to verify the conversion
print(df_bom_movies.head())

Non-numeric values in 'foreign_gross':
222    NaN
254    NaN
267    NaN
269    NaN
280    NaN
        ..
3382   NaN
3383   NaN
3384   NaN
3385   NaN
3386   NaN
Name: foreign_gross, Length: 1350, dtype: float64
                                         title studio  domestic_gross  \
0                                  Toy Story 3     BV     415000000.0   
1                   Alice in Wonderland (2010)     BV     334200000.0   
2  Harry Potter and the Deathly Hallows Part 1     WB     296000000.0   
3                                    Inception     WB     292600000.0   
4                          Shrek Forever After   P/DW     238700000.0   

   foreign_gross  year  
0    652000000.0  2010  
1    691300000.0  2010  
2    664300000.0  2010  
3    535700000.0  2010  
4    513900000.0  2010  


In [11]:
#the code below will give the % of missing values in the columns to guide my decision
df_bom_movies.isnull().sum()*100/len(df_bom_movies)

title              0.000000
studio             0.147623
domestic_gross     0.826690
foreign_gross     39.858282
year               0.000000
dtype: float64

In [12]:
#these are not huge % hence i will drop the rows with missing values
df_bom_movies.dropna(axis = 0,inplace = True)
df_bom_movies.isnull().sum()

title             0
studio            0
domestic_gross    0
foreign_gross     0
year              0
dtype: int64


# Next dataset is the rt.movies

In [13]:
df_rt
df_rt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [14]:
#i will drop the columns i don't need and remain with the ones i need for merging with the other datasets
df_rt.drop(['synopsis','director','writer','theater_date','dvd_date','currency','box_office',],axis = 1,inplace = True)
df_rt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       1560 non-null   int64 
 1   rating   1557 non-null   object
 2   genre    1552 non-null   object
 3   runtime  1530 non-null   object
 4   studio   494 non-null    object
dtypes: int64(1), object(4)
memory usage: 61.1+ KB


In [16]:
#Then check for duplicates in my remaining columns
df_rt.duplicated().value_counts()
#ther are no duplicates

False    1560
dtype: int64

In [18]:
#checking for null values
df_rt.isna().sum()

id            0
rating        3
genre         8
runtime      30
studio     1066
dtype: int64

In [19]:
#checking the percentage to make decision
df_rt.isnull().sum()*100/len(df_rt)

id          0.000000
rating      0.192308
genre       0.512821
runtime     1.923077
studio     68.333333
dtype: float64

In [20]:
#I am going to first work on the ratings column
df_rt['rating'].fillna('NA', inplace = True)

#Then Genre Column
df_rt['genre'].fillna('NA', inplace = True)

#Finally for Runtime Column
# Remove units of measurement from runtime column                                 
df_rt['runtime'] = [float(str(x).replace('minutes', '')) for x in df_rt['runtime']]

# Convert runtime column to float
df_rt['runtime'] = pd.to_numeric(df_rt['runtime'], errors='coerce')

# Inpute missing values in runtime column with median value
median_runtime = df_rt['runtime'].median()
df_rt['runtime'].fillna(median_runtime, inplace=True)

# Lets Change runtime column back to string representation of minutes
df_rt['runtime'] = df_rt['runtime'].astype(int).astype(str) + ' minutes'


df_rt.isnull().sum()*100/len(df_rt)

id          0.000000
rating      0.000000
genre       0.000000
runtime     0.000000
studio     68.333333
dtype: float64

In [21]:
df_rt

,id,rating,genre,runtime,studio
0,1,R,Action and Adventure|Classics|Drama,104 minutes,NaN
1,3,R,Drama|Science Fiction and Fantasy,108 minutes,Entertainment One
2,5,R,Drama|Musical and Performing Arts,116 minutes,NaN
3,6,R,Drama|Mystery and Suspense,128 minutes,NaN
4,7,NR,Drama|Romance,200 minutes,NaN
...,...,...,...,...,...
1555,1996,R,Action and Adventure|Horror|Mystery and Suspense,106 minutes,New Line Cinema
1556,1997,PG,Comedy|Science Fiction and Fantasy,88 minutes,Paramount Vantage
1557,1998,G,Classics|Comedy|Drama|Musical and Performing Arts,111 minutes,NaN
1558,1999,PG,Comedy|Drama|Kids and Family|Sports and Fitness,101 minutes,NaN


# Third dataset the movies budget

In [24]:
df_budget

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [25]:
df_budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [26]:
#does not seem to have null values
#what about duplicates?

In [30]:
df_budget.duplicated().value_counts()

False    5782
dtype: int64

In [31]:
df_budget['production_budget'] = df_budget['production_budget'].astype(str)
df_budget['domestic_gross'] = df_budget['domestic_gross'].astype(str)
df_budget['worldwide_gross'] = df_budget['worldwide_gross'].astype(str)
  
for r in ['production_budget', 'domestic_gross', 'worldwide_gross']:
    # Removes $ symbol and ,
    df_budget[r] = df_budget[r].str.replace('$', '').str.replace(',', '') 
    df_budget[r] = df_budget[r].astype(float) # Converts from object (str) to float
    # Divides by 100,000,000 to make it easier for the visualization
    df_budget[r] = (df_budget[r]).astype('int64') # Converts from float to int64
df_budget

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0
5778,79,"Apr 2, 1999",Following,6000,48482,240495
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0


# Merging the Datasets to see what we have

In [34]:
merged_data_one = pd.merge(df_bom_movies, df_rt, on='studio')
merged_data_one.head()
 

,title,studio,domestic_gross,foreign_gross,year,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime
0,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010,611,"Directed by Clint Eastwood, the mysterious dra...",R,Drama|Mystery and Suspense,Clint Eastwood,Brian Helgeland,"Oct 8, 2003","Jun 8, 2004",$,"88,800,000",137 minutes
1,Inception,WB,292600000.0,535700000,2010,611,"Directed by Clint Eastwood, the mysterious dra...",R,Drama|Mystery and Suspense,Clint Eastwood,Brian Helgeland,"Oct 8, 2003","Jun 8, 2004",$,"88,800,000",137 minutes
2,Clash of the Titans (2010),WB,163200000.0,330000000,2010,611,"Directed by Clint Eastwood, the mysterious dra...",R,Drama|Mystery and Suspense,Clint Eastwood,Brian Helgeland,"Oct 8, 2003","Jun 8, 2004",$,"88,800,000",137 minutes
3,Due Date,WB,100500000.0,111200000,2010,611,"Directed by Clint Eastwood, the mysterious dra...",R,Drama|Mystery and Suspense,Clint Eastwood,Brian Helgeland,"Oct 8, 2003","Jun 8, 2004",$,"88,800,000",137 minutes
4,Yogi Bear,WB,100200000.0,101300000,2010,611,"Directed by Clint Eastwood, the mysterious dra...",R,Drama|Mystery and Suspense,Clint Eastwood,Brian Helgeland,"Oct 8, 2003","Jun 8, 2004",$,"88,800,000",137 minutes


In [33]:
df_budget

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0
5778,79,"Apr 2, 1999",Following,6000,48482,240495
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0


In [35]:
#i will replace the name movie woth title to much with the merged dataset
df_budget.rename(columns={'movie': 'title'},inplace = 'True')
df_budget

,id,release_date,title,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0
5778,79,"Apr 2, 1999",Following,6000,48482,240495
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0


In [36]:
final_dataset = pd.merge(merged_data_one,df_budget, on = 'title')
final_dataset

,title,studio,domestic_gross_x,foreign_gross,year,id_x,synopsis,rating,genre,director,...,theater_date,dvd_date,currency,box_office,runtime,id_y,release_date,production_budget,domestic_gross_y,worldwide_gross
0,Inception,WB,292600000.0,535700000,2010,611,"Directed by Clint Eastwood, the mysterious dra...",R,Drama|Mystery and Suspense,Clint Eastwood,...,"Oct 8, 2003","Jun 8, 2004",$,"88,800,000",137 minutes,38,"Jul 16, 2010",160000000,292576195,835524642
1,Due Date,WB,100500000.0,111200000,2010,611,"Directed by Clint Eastwood, the mysterious dra...",R,Drama|Mystery and Suspense,Clint Eastwood,...,"Oct 8, 2003","Jun 8, 2004",$,"88,800,000",137 minutes,84,"Nov 5, 2010",65000000,100539043,211739043
2,Yogi Bear,WB,100200000.0,101300000,2010,611,"Directed by Clint Eastwood, the mysterious dra...",R,Drama|Mystery and Suspense,Clint Eastwood,...,"Oct 8, 2003","Jun 8, 2004",$,"88,800,000",137 minutes,62,"Dec 17, 2010",80000000,100246011,204774690
3,The Book of Eli,WB,94800000.0,62300000,2010,611,"Directed by Clint Eastwood, the mysterious dra...",R,Drama|Mystery and Suspense,Clint Eastwood,...,"Oct 8, 2003","Jun 8, 2004",$,"88,800,000",137 minutes,64,"Jan 15, 2010",80000000,94835059,158750817
4,The Town,WB,92200000.0,61800000,2010,611,"Directed by Clint Eastwood, the mysterious dra...",R,Drama|Mystery and Suspense,Clint Eastwood,...,"Oct 8, 2003","Jun 8, 2004",$,"88,800,000",137 minutes,92,"Sep 17, 2010",37000000,92186262,152566881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,The Florida Project,A24,5900000.0,NaN,2017,1399,Imagine the end of the world. Now imagine some...,R,Drama|Horror,Trey Edward Shults,...,"Jun 9, 2017","Sep 12, 2017",$,"13,985,117",91 minutes,11,"Oct 6, 2017",2000000,5904366,11295324
1339,Free Fire,A24,1800000.0,NaN,2017,1399,Imagine the end of the world. Now imagine some...,R,Drama|Horror,Trey Edward Shults,...,"Jun 9, 2017","Sep 12, 2017",$,"13,985,117",91 minutes,83,"Apr 21, 2017",10000000,1799322,3793739
1340,A Ghost Story,A24,1600000.0,NaN,2017,1399,Imagine the end of the world. Now imagine some...,R,Drama|Horror,Trey Edward Shults,...,"Jun 9, 2017","Sep 12, 2017",$,"13,985,117",91 minutes,86,"Jul 7, 2017",100000,1594798,2769782
1341,Hereditary,A24,44100000.0,35300000,2018,1399,Imagine the end of the world. Now imagine some...,R,Drama|Horror,Trey Edward Shults,...,"Jun 9, 2017","Sep 12, 2017",$,"13,985,117",91 minutes,56,"Jun 8, 2018",10000000,44069456,70133905
